In [8]:
import requests
import hashlib
import json
import copy

from kafka import KafkaProducer


In [9]:
r = requests.get('https://marvelsnapzone.com/getinfo/?searchtype=cards&searchcardstype=true')
cards = r.json()['success']['cards']


In [10]:
for card in cards:
    cid = card['cid']
    for tags in card['tags']:
        tags.update({ 'cid': cid })


In [11]:
producer = KafkaProducer(
    bootstrap_servers='kafka:9093',
    value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [12]:
for card in cards:
    for variant in card['variants']:
        producer.send('in-variants', variant)


In [13]:
for card in cards:
    for tag in card['tags']:
        producer.send('in-tags', tag)


In [14]:
for card in copy.deepcopy(cards):
    if 'variants' in card:
        card.pop('variants')
    if 'tags' in card:
        card.pop('tags')
    producer.send('in-cards', card)
